In [1]:
import modin.pandas as pd

In [2]:
dataset=pd.read_csv('milknew.csv')

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



In [ ]:
dataset.describe()

In [15]:
dataset.head()

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour,Grade
0,6.6,35,1,0,1,0,254,1.0
1,6.6,36,0,1,0,1,253,1.0
2,8.5,70,1,1,1,1,246,0.0
3,9.5,34,1,1,0,1,255,0.0
4,6.6,37,0,0,0,0,255,0.5


In [ ]:
dataset.isna().sum()

In [5]:
dataset['Grade'].replace({'high':1,"medium":0.5,"low":0},inplace=True)

In [6]:
dataset["Grade"][0:5]

0    1.0
1    1.0
2    0.0
3    0.0
4    0.5
Name: Grade, dtype: float64

In [25]:
X=dataset.drop("Grade",axis=1).values

In [26]:
Y=dataset["Grade"]

In [27]:
X.columns

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [28]:
from sklearn.preprocessing import StandardScaler
Scaler=StandardScaler()
X_scaled=pd.DataFrame(Scaler.fit_transform(X))

In [29]:
X_scaled.head()

,0,1,2,3,4,5,6
0,-0.021531,-0.914107,0.910503,-0.872962,0.699608,-0.982217,0.501600
1,-0.021531,-0.815035,-1.098294,1.145525,-1.429372,1.018105,0.269333
2,1.336564,2.553438,0.910503,1.145525,0.699608,1.018105,-1.356536
3,2.051351,-1.013180,0.910503,1.145525,-1.429372,1.018105,0.733867
4,-0.021531,-0.715962,-1.098294,-0.872962,-1.429372,-0.982217,0.733867


In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_scaled,Y,test_size=0.2,random_state=10)

In [31]:
print(X.shape, Y.shape, x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(1059, 7) (1059,) (847, 7) (212, 7) (847,) (212,)


In [32]:

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [33]:
model=DecisionTreeClassifier()
model.fit(x_train,y_train)
# prediction=model.predict(x_test[1:212])
# print(prediction)

ValueError: Unknown label type: 'continuous'

In [ ]:
metrics.accuracy_score(prediction,y_test[1:212])